In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Basics
import tensorflow as tf
import xarray as xr

# Helpful
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *  # CRPS metrics
from src.utils.data_split import *  # Splitting data into X and y
from src.utils.drn_make_X_array import *  # Import make train array functions (make_X_array)
from src.models.EMOS import *  # EMOS implementation
from src.models.DRN.DRN_model import *  # DRN implementation
from src.models.EMOS_global.EMOS_global_load_score import *  # Load EMOS_global_scores
from src.models.EMOS_global.EMOS_global_load_model import *  # Load EMOS_global_models
import data.raw.load_data_raw as ldr  # Load raw data
import data.processed.load_data_processed as ldp  # Load processed data normed
import data.processed.load_data_processed_denormed as ldpd  # Load processed data denormed
from src.models.CRPS_baseline.CRPS_load import *  # Load CRPS scores

2023-05-31 21:38:25.446614: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-31 21:38:25.446643: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 0. Set up

In [3]:
lead_time = 0

<IPython.core.display.Javascript object>

### 1. Load Data

In [4]:
dat_train_denorm = ldpd.load_data_all_train_proc_denorm()
dat_test_denorm = ldpd.load_data_all_test_proc_denorm()

<IPython.core.display.Javascript object>

### 2. Data Split

In [5]:
dat_X_train_lead_all_denorm, dat_y_train_lead_all_denorm = split_var_lead(
    dat_train_denorm
)
dat_X_test_lead_all_denorm, dat_y_test_lead_all_denorm = split_var_lead(dat_test_denorm)

<IPython.core.display.Javascript object>

### 3. Data preparation

Idea: X_train constructed by flattening the value arrays and concatenating them together, so that in the end I get the shape (n, n_variables), keep in mind we use both mean and std of each variabel for training. So in total we will have 10 / 12, depending on which variable. Important, for drn we'll also need the embeddings, hence the idea to keep another flattened array with shape (n, 1) with the complementing embedding (1 - 15600).

In [6]:
# Make X train data into NN feedable array
drn_X_train_0_array, drn_embedding_train_0_array = make_X_array(
    dat_X_train_lead_all_denorm, 0
)

<IPython.core.display.Javascript object>

In [7]:
# Make X test data into NN feedable array
drn_X_test_0_array, drn_embedding_test_0_array = make_X_array(
    dat_X_test_lead_all_denorm, 0
)

<IPython.core.display.Javascript object>

In [8]:
# Put y values into correct form
t2m_y_train = dat_y_train_lead_all_denorm[2][0].values.flatten()
t2m_y_test = dat_y_test_lead_all_denorm[2][0].values.flatten()

<IPython.core.display.Javascript object>

### 4. Model SetUp

In [30]:
hidden_layer = []
emb_size = 3
max_id = 15600 - 1

<IPython.core.display.Javascript object>

In [31]:
drn_0_model = build_emb_model(12, 2, hidden_layer, emb_size, max_id, compile=True)

<IPython.core.display.Javascript object>

### 5. Train DRN for lead_time 0, t2m

In [32]:
drn_0_model.fit(
    [drn_X_train_0_array, drn_embedding_train_0_array],
    t2m_y_train,
    epochs=10,
    batch_size=8192,
    validation_data=[[drn_X_test_0_array, drn_embedding_test_0_array], t2m_y_test],
)

Epoch 1/10
2722/2722 [==============================] - 10s 3ms/step - loss: 50.8725 - val_loss: 2.8323
Epoch 2/10
2722/2722 [==============================] - 9s 3ms/step - loss: 2.6320 - val_loss: 2.6045
Epoch 3/10
2722/2722 [==============================] - 9s 3ms/step - loss: 2.3782 - val_loss: 2.3234
Epoch 4/10
2722/2722 [==============================] - 9s 3ms/step - loss: 2.0636 - val_loss: 1.9907
Epoch 5/10
2722/2722 [==============================] - 8s 3ms/step - loss: 1.7410 - val_loss: 1.6719
Epoch 6/10
2722/2722 [==============================] - 8s 3ms/step - loss: 1.4683 - val_loss: 1.4304
Epoch 7/10
2722/2722 [==============================] - 8s 3ms/step - loss: 1.2776 - val_loss: 1.2681
Epoch 8/10
2722/2722 [==============================] - 8s 3ms/step - loss: 1.1546 - val_loss: 1.1526
Epoch 9/10
2722/2722 [==============================] - 8s 3ms/step - loss: 1.0717 - val_loss: 1.0748
Epoch 10/10
2722/2722 [==============================] - 9s 3ms/step - loss: 1.0

<IPython.core.display.Javascript object>

In [27]:
drn_X_train_0_array.shape

(22292400, 12)

<IPython.core.display.Javascript object>

In [18]:
EMOS_global_t2m_score = EMOS_global_load_score("t2m")

<IPython.core.display.Javascript object>

In [25]:
EMOS_global_t2m_score[29].mean()

1.2656613124592335

<IPython.core.display.Javascript object>

In [28]:
4096 * 2

8192

<IPython.core.display.Javascript object>